In [1]:
from sklearn.datasets import make_classification

In [2]:
make_classification?

In [3]:
base_dataset = make_classification(
    n_samples=10_000, 
    n_features=30, 
    n_informative=10,
    n_clusters_per_class=2,
    n_classes=4)

In [4]:
x, y = base_dataset

In [5]:
base_classes = []

for i in range(4):
    base_classes.append(x[y == i])

In [6]:
import numpy as np

num_points = 5_000
class1_dist = [.5, .5, 0, 0]
class2_dist = [0, .2, .6, .2]

def make_var_len_feature_point(dist):
    feature_sets = []
    previous_feature_set = np.zeros((1, 30))
    num_features = np.random.randint(3, 11)
    for _ in range(num_features):
        # choose which distribution the customer comes from
        base_class = np.random.choice([0, 1, 2, 3], 1, p=dist)
        base_class_points = base_classes[base_class[0]]
        feature_set_idx = np.random.choice(base_class_points.shape[0], 1)
        previous_feature_set += base_class_points[feature_set_idx]
        feature_sets.append(previous_feature_set)
        
    for _ in range(10 - num_features):
        feature_sets.append(np.zeros((1, 30)))

    return np.concatenate(feature_sets)[np.newaxis, :, :]


class1_points = []
for _ in range(num_points):
    class1_points.append(
        make_var_len_feature_point(class1_dist))
class1_points = np.concatenate(class1_points)
    
class2_points = []
for _ in range(num_points):
    class2_points.append(
        make_var_len_feature_point(class2_dist))
class2_points = np.concatenate(class2_points)

In [7]:
class2_points.shape

(5000, 10, 30)

# NORMALIZE

In [8]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(
            class1_points.shape[0], batch_size // 2)
        batch_x = np.concatenate([
            class1_points[batch_idx],
            class2_points[batch_idx],
        ])
        batch_y = np.concatenate([
            np.zeros(batch_size // 2),
            np.ones(batch_size // 2),
        ])
        yield ({'numeric_inputs': batch_x}, 
               {'output': batch_y})

In [9]:
for xx in bootstrap_sample_generator(32):
    break

In [10]:
xx[0]['numeric_inputs'].shape

(32, 10, 30)

In [13]:
p = .1

In [11]:
import tensorflow as tf

In [15]:
inputs = tf.keras.layers.Input((10, 30), name='numeric_inputs')

In [18]:
x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(10))(x)


In [19]:
x.shape

TensorShape([None, 20])

In [20]:
# bonus
x = tf.keras.layers.RepeatVector(10)(x)
x = tf.keras.layers.Concatenate()([inputs, x])

x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(10))(x)

In [21]:
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

In [22]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
numeric_inputs (InputLayer)     [(None, 10, 30)]     0                                            
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 10, 30)       0           numeric_inputs[0][0]             
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20)           2520        dropout_2[0][0]                  
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 10, 20)       0           bidirectional_1[0][0]            
______________________________________________________________________________________________

In [24]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=5,
    max_queue_size=10,
)

Epoch 1/5
312/312 [==============================] - 10s 32ms/step - loss: 0.6054 - accuracy: 0.6782
Epoch 2/5
312/312 [==============================] - 4s 13ms/step - loss: 0.4494 - accuracy: 0.7974
Epoch 3/5
312/312 [==============================] - 4s 13ms/step - loss: 0.4074 - accuracy: 0.8184
Epoch 4/5
312/312 [==============================] - 4s 13ms/step - loss: 0.3730 - accuracy: 0.8363
Epoch 5/5
312/312 [==============================] - 4s 13ms/step - loss: 0.3598 - accuracy: 0.8441
